<a href="https://colab.research.google.com/github/raquel-ldn/bcs-thesis/blob/main/ArgumentMiningTests.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Formatting

In [123]:
from IPython.display import HTML, display

def set_css():
  display(HTML('''
  <style>
    pre {
        white-space: pre-wrap;
    }
  </style>
  '''))
get_ipython().events.register('pre_run_cell', set_css)

Install Webscrapper
ToDo: Try different scrapers i.e. scraphero

In [124]:
 !npm install -g npm
 !npm i -g amazon-buddy

/tools/node/bin/npx -> /tools/node/lib/node_modules/npm/bin/npx-cli.js
/tools/node/bin/npm -> /tools/node/lib/node_modules/npm/bin/npm-cli.js
+ npm@6.14.11
updated 1 package in 6.734s
npm WARN deprecated request@2.88.2: request has been deprecated, see https://github.com/request/request/issues/3142
npm WARN deprecated request-promise@4.2.6: request-promise has been deprecated because it extends the now deprecated request package, see https://github.com/request/request/issues/3142
npm WARN deprecated cheerio-select-tmp@0.1.1: Use cheerio-select instead
npm WARN deprecated har-validator@5.1.5: this library is no longer supported
/tools/node/bin/amazon-buddy -> /tools/node/lib/node_modules/amazon-buddy/bin/cli.js
+ amazon-buddy@2.2.26
updated 1 package in 3.3s


Install NLP Python libs


*   `sentence-transformers` for BERT transformations
*   `umap-learn` for dimension reduction algorithm 
*   `hdbscan` for clustering and creation of areas with high denisty hopefully our topics
*   `gensim` for topic modelling without BERT
*   `spacy` for data cleaning, stop words, lemanisation etc for data 
*   `nltk` lemanisation



ToDO: try scikit-learn for umap



In [125]:
!pip install sentence-transformers
!pip install umap-learn
!pip install hdbscan
!pip install --upgrade gensim
#!pip install -U spacy
!pip install nltk

Requirement already up-to-date: gensim in /usr/local/lib/python3.6/dist-packages (3.8.3)


Downloading Claim and Premise Indicators If data already on disk force overwrite
<br><b>WIP adjust list to review data


In [126]:
!wget https://raw.githubusercontent.com/raquel-ldn/argument_BERT/development/preprocessing/claim_indicator.txt -O claim_indicator.txt
!wget https://raw.githubusercontent.com/raquel-ldn/argument_BERT/master/preprocessing/premise_indicator.txt -O premise_indicator.txt

--2021-01-14 20:23:36--  https://raw.githubusercontent.com/raquel-ldn/argument_BERT/development/preprocessing/claim_indicator.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 540 [text/plain]
Saving to: ‘claim_indicator.txt’

claim_indicator.txt 100%[===================>]     540  --.-KB/s    in 0s      

2021-01-14 20:23:36 (31.0 MB/s) - ‘claim_indicator.txt’ saved [540/540]

--2021-01-14 20:23:36--  https://raw.githubusercontent.com/raquel-ldn/argument_BERT/master/preprocessing/premise_indicator.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200

Execute  scraper for ASIN `B0141712VI` and return 2000 reviews. write the output of the command into a file called `console.log`

amazon-buddy creates a csv file with the name of the ASIN and a random number the file name is written into `console.log`

Read the filename from `console.log` and extract the ASIN. Use the ASIN to change to final name `B0141712VI.csv` for later use.



In [127]:
%env ASIN=B07JVGB9SH
!amazon-buddy reviews $ASIN --country 'GB' -n 2000 > console.log
!cat console.log
%mv `cat console.log | tr ' ' '\n' | tail -n1`.csv  $ASIN'.csv'
!wc -l $ASIN'.csv'

env: ASIN=B07JVGB9SH
⠋ Amazon Scraper Started⠙ Amazon Scraper Started⠹ Amazon Scraper Started⠸ Amazon Scraper Started⠼ Amazon Scraper Started⠴ Amazon Scraper Started⠦ Amazon Scraper Started⠧ Amazon Scraper Started⠇ Amazon Scraper Started⠏ Amazon Scraper Started⠋ Amazon Scraper Started⠙ Amazon Scraper Started⠹ Amazon Scraper Started⠸ Amazon Scraper Started⠼ Amazon Scraper Started⠴ Amazon Scraper Started⠦ Amazon Scraper Started⠧ Amazon Scraper Started⠇ Amazon Scraper Started⠏ Amazon Scraper Started⠋ Amazon Scraper Started⠙ Amazon Scraper Started⠹ Amazon Scraper Started⠸ Amazon Scraper Started⠼ Amazon Scraper Started⠴ Amazon Scraper Started⠦ Amazon Scraper Started⠧ Amazon Scraper Started⠇ Amazon Scraper Started⠏ Amazon Scraper Started⠋ Amazon Scraper Started⠙ Amazon Scraper Started⠹ Amazon Scraper Started⠸ Amazon Scraper Started⠼ Amazon Scraper Started⠴ Amazon Scraper Started⠦ Amazon Scraper Started⠧ Amazon Scraper Started⠇ Amazon Scraper Started⠏ Amazon Scraper Started⠋ Amazon Scraper St

This is Work in Progress ETL
<br><b> Main tasks:</b>
<br> filtering sentences containing at least one claim and one premise
<br> improving text quality through correcting and cleaning
<br> 

<br><b> WIP select senteces were claim and premise are spread across a pair or multiple senteces.

In [128]:
import nltk
import re
import csv
import os
import re  

nltk.download('punkt')
PREMISE_FILE = 'premise_indicator.txt'
CLAIM_FILE = 'claim_indicator.txt'
FILE = os.environ['ASIN'] + '.csv'

def read_review_data(csv_file_name,column):
  """
  function to detect to read CSV file and extract one column of review
  :param filename column:
  :return: list object containing review data
  """
  list_of_review_sentences = []
  csv_file = open(csv_file_name)
  read_csv = csv.reader(csv_file, delimiter=",")
  
  #skip header
  next(read_csv)
  for row in read_csv:
    review_text = row[column]
    list_of_review_sentences.append(review_text)
 
  return  list_of_review_sentences

def read_key_words(file):
    """Reads list of words in file, one keyword per line"""

    return [line.rstrip('\n') for line in open(file)]

def check_indicators(sentence, word_list):
    """
    function to detect the presence of argument keywords in a sentence
    :param full sentence:
    :return: True if sentence contains keyword
    """

    for indicator in word_list:
      if re.search(r"\b" + re.escape(indicator.lower()) + r"\b",sentence.lower()):
        return 1
    return 0


def check_topic(sentence, string):
    """
    function to detect the presence of topic keywords in a sentence
    :param full sentence:
    :return: True if sentence contains keyword
    """
    if re.search(r"\b" + re.escape(string.lower()) + r"\b",sentence.lower()):
      return 1
    return 0

def copy_list_elements(src_list, dest_list):
  """
  function to copy elements of a list into an existing list
  :param:  
  :param:
  :return: True if sentence contains keyword
  """
  for element in src_list:
    dest_list.append(element)

def correct_punctuation(line_of_review_data,punctuation_char):
  """
  function to add a space after every '.' and separate words containing '/' elements of a list into an existing list
  :param review data to be corrected:  
  :return: Corrected text data
  """
  # add a space after a dot
  look_for = rf"\{punctuation_char}(?!\s|$)"
  replace_with=rf"{punctuation_char} "
  # add a space before and after '/'
  #rx2 = r'\b[//:]\b'
  corrected_review_data = re.sub(look_for, replace_with, line_of_review_data)
  #line_of_review_data = re.sub(rx2, " // ", line_of_review_data)

  return corrected_review_data

def extract_arguments(doc):
  argumentative_setences = []
  claim_setences         = []
  premise_setences       = []

  #claim_counter = 0
  #premise_counter = 0
  
  for review_text in docs:  
    # clean up not well formated reviews  
    review_text = correct_punctuation(review_text, ".")
    review_text = correct_punctuation(review_text, "!")
    review_text = correct_punctuation(review_text, "/")
    # this gives us a list of sentences
    sent_text   = nltk.sent_tokenize(review_text) 
    # now loop over each sentence and tokenize it separately
    for sentence in sent_text:
      if check_indicators(sentence,claim_list) and check_indicators(sentence,premise_list):
        argumentative_setences.append(sentence)
        # print(sentence)
      elif check_indicators(sentence,claim_list):
        claim_setences.append(sentence)
      elif check_indicators(sentence,premise_list):
        premise_setences.append(sentence)
  return argumentative_setences, claim_setences, premise_setences 

premise_list = read_key_words(PREMISE_FILE)
claim_list = read_key_words(CLAIM_FILE)

argumentative_setences = []
claim_setences         = []
premise_setences       = []
argumentative_docs     = []

docs = read_review_data(FILE,column=9)
argumentative_setences, claim_setences, premise_setences = extract_arguments(docs)

print("################ Arguments: " + str(len(argumentative_setences)))
print("################ Claims: "  + str(len(claim_setences)))
print("################ Premise: " + str(len(premise_setences)))


copy_list_elements(argumentative_setences,argumentative_docs)
'''we are keeping this information in order to extract claims and premises potentially spread over multiple sentences.'''
copy_list_elements(claim_setences,argumentative_docs)
copy_list_elements(premise_setences,argumentative_docs)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
################ Arguments: 293
################ Claims: 401
################ Premise: 1905


Lemmatizsation in an attempt to improve cosine vectors similarity <br>

In [129]:
from nltk.stem import PorterStemmer 
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet

nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
lemmatizer = WordNetLemmatizer()
ps = PorterStemmer()

def nltk2wn_tag(nltk_tag):
  """
  function to convert nltk pos tag into wordnet tag which can feed into WordNetLemmatizer
  :param nltk tag:
  :return: the corresponding wordnet tag
  """
  if nltk_tag.startswith('J'):
    return wordnet.ADJ
  elif nltk_tag.startswith('V'):
    return wordnet.VERB
  elif nltk_tag.startswith('N'):
    return wordnet.NOUN
  elif nltk_tag.startswith('R'):
    return wordnet.ADV
  else:                    
    return None

def lemmatize_sentence(sentence):
  """
  function to convert all words in a sentence to there corresponding lemmatized version
  :param sentence:
  :return: the lemmatized sentence
  """
  nltk_tagged = nltk.pos_tag(nltk.word_tokenize(sentence)) 
  wn_tagged = map(lambda x: (x[0], nltk2wn_tag(x[1])), nltk_tagged)
  res_words = []
  for word, tag in wn_tagged:
    if tag is None:                        
      res_words.append(word)
    else:
      res_words.append(lemmatizer.lemmatize(word, tag))

  return " ".join(res_words)
'''
def stemm_sentence(sentence):
  res_words = []
  for word in sentence:
        #print(ps.stem(word))
        res_words.append(ps.stem(word))
  return "".join(res_words)
'''
lemmatize_docs = []
for sentence in argumentative_docs:
  lemmatize_docs.append(lemmatize_sentence(sentence))

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Creating BERT Embeddings using `SentenceTransformer` <br>
[We are using pre-trained model](https://docs.google.com/spreadsheets/d/14QplCdTCDwEmTqrn1LH4yrbKvdogK4oQvYO1K1aPR5M/edit#gid=0)
  'paraphrase-distilroberta-base-v1'



In [138]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('paraphrase-distilroberta-base-v1')
embeddings = model.encode(lemmatize_docs, show_progress_bar=True)

The following functions identifies cluster in our BERT Embeddgins and extracts topics from the cluster using TDF IDF<br>
<b>WIP add review specific stop words</b>

In [166]:
#import umap
import hdbscan
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction import text 
'''
umap_embeddings = umap.UMAP(n_neighbors=15, n_components=5,metric='cosine').fit_transform(embeddings)

cluster = hdbscan.HDBSCAN(min_cluster_size=10,
                          metric='euclidean',                      
                          cluster_selection_method='eom').fit(umap_embeddings)
'''
cluster = hdbscan.HDBSCAN(algorithm='best', alpha=1.0, approx_min_span_tree=True,
    gen_min_span_tree=False, leaf_size=40,
    metric='manhattan', min_cluster_size=3, min_samples=None, p=None).fit(embeddings)

def label_data_frame(docs): 
  """
  function to convert a list of review text into a panda data frame
  :param: docs a list of review text
  :return: docs data frame docs_per_topic
  """                 
  docs_df           = pd.DataFrame(docs, columns=["Doc"])
  docs_df['Topic']  = cluster.labels_
  docs_df['Doc_ID'] = range(len(docs_df))
  docs_per_topic    = docs_df.groupby(['Topic'], as_index = False).agg({'Doc': ' '.join})
  # print(docs_per_topic)
  # print(docs_per_topic.Topic.values)
  return docs_df, docs_per_topic

def c_tf_idf(documents, m, ngram_range=(1, 1)):
  """
  function to convert a list of review text into a panda data frame
  :param: docs a list of review text
  :return: docs data frame docs_per_topic
  """ 
  #additional stop words 
  stop_words  = text.ENGLISH_STOP_WORDS.union(
      ["mandatory","reasonable","excellent",
       "wonderful","pretty","family","fantastic",
       "great","happy","good","order","delivery",
       "shipping","buy","husband","wife", "search",
       "mainly","recommended","highly","overly",
       "come","christmas","andi","love","brother","sister",
       "son","daughter","birthday","gift","xmas","got","nephew"
       "child","nephew","niece"])
  

  #create a CountVectorizer to count the number of words (term frequency), limit vocabulary size, apply stop words 
  counter     = CountVectorizer(ngram_range=ngram_range, stop_words=stop_words).fit(documents)
  word_count_vector = counter.transform(documents)
  
  # settings that you use for count vectorizer will go here 
  tfidf_vectorizer=TfidfVectorizer(ngram_range=ngram_range, stop_words=stop_words,use_idf=True) 
 
  # just send in all your docs here 
  tfidf_vectorizer_vectors=tfidf_vectorizer.fit_transform(documents)
 
  # just send in all your docs here
  fitted_vectorizer=tfidf_vectorizer.fit(docs)
  tfidf_vectorizer_vectors=fitted_vectorizer.transform(docs)
  
  transformer = counter.transform(documents).toarray()

  w = transformer.sum(axis=1)
  tf = np.divide(transformer.T, w)
  sum_transformer = transformer.sum(axis=0)
  idf = np.log(np.divide(m, sum_transformer)).reshape(-1, 1)
  tf_idf = np.multiply(tf, idf)

 # print(tf_idf)
 # print(counter)
#  print(tfidf_vectorizer)
 # print(tfidf_vectorizer_vectors.)

  return tf_idf, counter
  #return tfidf_vectorizer_vectors, tfidf_vectorizer

def extract_top_n_words_per_topic(tf_idf, count, docs_per_topic, n=20):
  words = count.get_feature_names()
  labels = list(docs_per_topic.Topic)
  tf_idf_transposed = tf_idf.T
  indices = tf_idf_transposed.argsort()[:, -n:]
  top_n_words = {
      label: [(words[j], tf_idf_transposed[i][j]) for j in indices[i]][::-1] for i, label in enumerate(labels)
  }
  return top_n_words

def extract_topic_sizes(df):
  topic_sizes = (df.groupby(['Topic'])
    .Doc
    .count()
    .reset_index()
    .rename({"Topic": "Topic", "Doc": "Size"}, axis='columns')
    .sort_values("Size", ascending=False))
  return topic_sizes
  
docs_df, docs_per_topic = label_data_frame(lemmatize_docs)
tf_idf, count = c_tf_idf(docs_per_topic.Doc.values, m=len(docs))

top_n_words = extract_top_n_words_per_topic(tf_idf, count, docs_per_topic, n=10)
topic_sizes = extract_topic_sizes(docs_df); 
topic_sizes.head(10)
#topic_sizes.head(20)

,Topic,Size
0,-1,2305
29,28,20
31,30,18
13,12,12
42,41,12
35,34,11
37,36,10
32,31,10
20,19,9
53,52,9


In [177]:

top_n_words[31]

[('connect', 0.2603318083320586),
 ('phone', 0.18729910539688174),
 ('wifi', 0.17825157348103216),
 ('port', 0.17587210905913503),
 ('allow', 0.1383907181647809),
 ('mode', 0.11071257453182472),
 ('usb', 0.08951806379963242),
 ('telly', 0.0885895589015473),
 ('remotely', 0.08325449168959777),
 ('video', 0.08024090281692114)]

In [162]:
#@title Select a Topic/feature
#@markdown Tye here a word from above topic list
string_type = 'wifi'  #@param {type: "string"}
#@markdown ---

def get_topical_sentences(topic, sentences):
  for sentence in sentences:
    if check_topic(sentence,topic):
      print(sentence)

get_topical_sentences(string_type,argumentative_setences)




WiFi requires a dedicated link to mobile and slow at transfer so unless you use it as a remote webcam to a phone it's a bit pointless.
I have used cheaper versions of this type of camera and can honestly say that this Crosstour 4K 20MP Action Camera WiFi EIS Waterproof 40M with External Microphone and Remote Control is extremely good value and as good in my opinion as many much more expensive products on the market.
However my favourite part about this camera is once connected to wifi you can take pictures/ videos from your phone, as you can see the cameras vision whilst connected to your phone.
It came with spare batteries , external microphone , and every type of mount you can imagine , the wifi was simple to connect to , however i chose to use usb for ease for myself.
I also only expected that the companion app would work if both your phone and the camera were connected to an already existing WiFi network but the camera actually broadcasts it own WiFi so no matter where you are you 